In [36]:
from typing import Tuple
import numpy as np

def simplex_method(c: np.ndarray, A: np.ndarray, b: np.ndarray) -> Tuple[np.ndarray, float]:
    # Obter o número de restrições (linhas) e o número de variáveis (colunas)
    cols, num_values = A.shape
    num_slack_vars = cols
    
    # Adicionar variáveis de folga para converter as restrições para a forma padrão
    A = np.hstack([A, np.eye(cols)])
    c = np.hstack([c, np.zeros(num_slack_vars)])
    b = b.copy().astype(float)
    
    # Inicializar a base com as variáveis de folga
    basis = [i + num_values for i in range(cols)]
    
    # Loop principal do método simplex
    while True:
        # Calcular cb (custo das variáveis básicas)
        cb = np.array([c[j] for j in basis])
        # Calcular a matriz dos coeficientes reduzidos z
        z = cb @ np.linalg.inv(A[:, basis]) @ A
        # Calcular cj - zj para determinar a variável de entrada
        cj_zj = c - z
        
        # Se todos os valores de cj_zj forem negativos ou zero, a solução atual é ótima
        if np.all(cj_zj <= 0):
            break

        # Determinar a variável de entrada (maior valor positivo em cj - zj)
        entering = np.argmax(cj_zj).astype(int)
        # Calcular as razões para determinar a variável de saída
        ratios = np.array([b[i] / A[i, entering] if A[i, entering] > 0 else np.inf for i in range(cols)])
        exiting = np.argmin(ratios)
        # Atualizar a base
        basis[exiting] = entering
        
        # Pivotear em torno da variável de entrada e saída
        b[exiting] /= A[exiting, entering]
        A[exiting, :] /= A[exiting, entering]
        for col in range(cols):
            if col == exiting:
                continue
            ratio = A[col, entering]
            b[col] -= ratio * b[exiting]
            A[col, :] -= ratio * A[exiting, :]

    # Extrair os valores de todas as variáveis do resultado
    variables = np.zeros(num_values + num_slack_vars)
    for i, var_idx in enumerate(basis):
        variables[var_idx] = b[i]

    # Calcular o valor da função objetivo
    objective = float(cb @ b)
    return variables, objective


In [37]:
from scipy.optimize import linprog

def solve_simplex(c: np.ndarray, A: np.ndarray, b: np.ndarray):
    solution, value = simplex_method(c, A, b)
    print(f'O - Solution: {solution}, Optimal value: {value}')

def solve_scipy(c: np.ndarray, A: np.ndarray, b: np.ndarray):
    # Note: linprog tries to minimize, not maximize, so we negate c
    bounds = [(0, None), (0, None)]

    result = linprog(-c, A_ub=A, b_ub=b, bounds=bounds)

    result_x = result.x
    result_fun = -result.fun  # We negate to get the maximized value
    result_s = result.slack
    result_concat = np.concatenate([result_x, result_s])
    print(f'S - Solution: {result_concat}, Optimal value: {result_fun}')
    # print(result)

In [38]:
# Example problem
# Maximize z = 3x1 + 2x2
# Subject to:
# 2x1 + x2 <= 8
# x1 + 2x2 <= 6
# x1, x2 >= 0

# c is the coefficients of the objective function
# A is the matrix of coefficients of the constraints
# b is the vector of values on the right side of the constraints
c = np.array([5, 3.])
A = np.array([[2, 3.], [2, 1], [1, -1]])
b = np.array([15, 9., 3])

solve_simplex(c, A, b)
solve_scipy(c, A, b)

O - Solution: [3. 3. 0. 0. 3.], Optimal value: 24.0
S - Solution: [3. 3. 0. 0. 3.], Optimal value: 24.0


In [39]:
c = np.array([7, 6])
A = np.array([[2, 4], [3, 2]])
b = np.array([16, 12])

solve_simplex(c, A, b)
solve_scipy(c, A, b)

O - Solution: [2. 3. 0. 0.], Optimal value: 32.0
S - Solution: [2. 3. 0. 0.], Optimal value: 32.0


In [40]:
c = np.array([5, 7])
A = np.array([[7, 3], [2, 5], [3, -1]])
b = np.array([15, 30, 10])

solve_simplex(c, A, b)
solve_scipy(c, A, b)

O - Solution: [ 0.  5.  0.  5. 15.], Optimal value: 35.0
S - Solution: [ 0.  5.  0.  5. 15.], Optimal value: 35.0
